In [4]:
import pandas as pd
import numpy as np

from collections import Counter
from itertools import product
import random

I tried a mathematical approach but failed. I want to try the problem using [Monte Carlo simulation][1] or [Markov chain][2], when I have the time.

  [1]: https://en.wikipedia.org/wiki/Monte_Carlo_method
  [2]: https://en.wikipedia.org/wiki/Markov_chain

In [11]:
# special squares
GO = 0
JAIL = 10
GO_TO_JAIL = 30
COMMUNITY_CHESTS = {2, 17, 33}
CHANCES = {7, 22, 36}
RAILROADS = [5, 15, 25, 35]
UTILITIES = {12, 28}
C1 = 11
E3 = 24
H2 = 39

# SET SIDES OF DIE
sides_of_die = 6

# die roll probabilities
sum_of_dice = [i + j + 2 for i,j in product(range(sides_of_die), range(sides_of_die))]
dice_probs = {i: sum([j == i for j in sum_of_dice])/(sides_of_die*sides_of_die) for i in range(2,sides_of_die*2 + 1)}

# community chest probabilities
cc_probs = np.array([1/16, 1/16, 14/16])

# chance probabilities
ch_probs = np.array([1/16, 1/16, 1/16, 1/16, 1/16, 1/16, 2/16, 1/16, 1/16, 6/16])

In [40]:
num_trials = 5*10**6
dice_choice = np.random.choice(np.array(list(dice_probs.keys()), dtype = int), p = np.array(list(dice_probs.values())))

current_square = 17
cc_choice = np.random.choice(np.array([GO, JAIL, current_square], dtype = int), p = cc_probs)

current_square = 36
next_railroad = min(min([r if r > current_square else 40 for r in RAILROADS]), 5 if current_square >= 35 else 40)
next_utility = min(min([u if u > current_square else 40 for u in UTILITIES]), 12 if current_square >= 28 else 40)
ch_choice = np.random.choice(
    np.array(
        [GO, JAIL, C1, E3, H2, RAILROADS[0], next_railroad, next_utility, current_square - 3, current_square], 
        dtype = int
    ), 
    size = (num_trials,),
    p = ch_probs)

Counter(ch_choice)

Counter({36: 1876049,
         5: 935798,
         24: 312952,
         33: 312773,
         10: 312720,
         39: 312570,
         0: 312536,
         12: 312340,
         11: 312262})

In [165]:
board = np.zeros(40, int)

for start_square in range(40):
    for roll in sum_of_dice.keys():
        next_square = (start_square + roll) % 40
        prob = sum_of_dice[roll]

        if next_square == GO_TO_JAIL:
            board[JAIL] += 16*prob
            continue

        # cases for community chest
        if next_square in COMMUNITY_CHESTS:
            board[JAIL] += prob
            board[GO] += prob
            board[next_square] += 14*prob
            continue
        
        # cases for chance
        if next_square in CHANCES:
            board[JAIL] += prob
            board[GO] += prob
            board[RAILROADS[0]] += prob
            board[min([r if r > next_square else 40 for r in RAILROADS]) if next_square < 35 else 5] += 2*prob
            board[min([u if u > next_square else 40 for u in UTILITIES]) if next_square < 28 else 5] += prob
            board[C1] += prob
            board[E3] += prob
            board[H2] += prob
            board[next_square - 3] += prob
            board[next_square] += 6*prob
            continue

        # not jail, not community chest, not chance
        board[next_square] += 16*prob

In [166]:
board / (40 * 16 * sides_of_die**2)

array([0.034375 , 0.025    , 0.021875 , 0.025    , 0.0265625, 0.034375 ,
       0.025    , 0.009375 , 0.025    , 0.025    , 0.059375 , 0.0296875,
       0.0265625, 0.025    , 0.025    , 0.028125 , 0.025    , 0.021875 ,
       0.025    , 0.0265625, 0.025    , 0.025    , 0.009375 , 0.025    ,
       0.0296875, 0.028125 , 0.025    , 0.025    , 0.0265625, 0.025    ,
       0.       , 0.025    , 0.025    , 0.0234375, 0.025    , 0.025    ,
       0.009375 , 0.025    , 0.025    , 0.0296875])

In [163]:
np.sum(board / (40 * 16 * sides_of_die**2))

1.0